In [1]:
!pip install tiktoken
!pip install triton
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.6 MB/s eta 0:00:00


In [2]:
# # scan your code and find the ugly number , we want to get the GPU freindly code

# from dataclasses import dataclass
# import inspect
# import torch
# import torch.nn as nn
# from torch.nn import functional as F

# # prefix tokens
# import tiktoken
# import os
# import json
# from tokenizers import ByteLevelBPETokenizer

# torch.set_float32_matmul_precision("high")
# # # Set the CUDA_VISIBLE_DEVICES environment variable
# # # os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Use GPU 0
# # print("Available GPUs:", torch.cuda.device_count())
# # # import os
# # # import torch

# # # Clear CUDA environment setting
# # # os.environ["CUDA_VISIBLE_DEVICES"] = ""


# # # Reinitialize CUDA to clear prior states
# # if torch.cuda.is_available():
# #     torch.cuda.empty_cache()

# # # Attempt to do the autodetect of the device(GPU or CPU)

# # device = "cpu"
# # torch.manual_seed(1337)
# # if torch.cuda.is_available():
# #     device = "cuda"
# #     torch.cuda.manual_seed(1337)

# # elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
# #     device = "mps"

# # print(f"using device : {device}")


# # print("Available GPUs:", torch.cuda.device_count())


# class CausalSelfAttention(nn.Module):
#     def __init__(self, config) -> None:
#         super().__init__()
#         assert config.n_embd % config.n_head == 0

#         self.c_attn = nn.Linear(config.n_embd, config.n_embd * 3)
#         self.n_head = config.n_head
#         self.c_proj = nn.Linear(config.n_embd, config.n_embd)
#         self.n_embd = config.n_embd
#         self.c_proj.GPT_SCALE = 1
#         self.register_buffer(
#             "bias",
#             torch.tril(torch.ones(config.block_size, config.block_size)).view(
#                 1, 1, config.block_size, config.block_size
#             ),
#         )

#     def forward(self, x):
#         B, T, C = (
#             x.size()
#         )  # batch size, sequence length, embedding dimensionality (n_embd)
#         # calculate query, key, values for all heads in batch and move head forward to be the batch dim
#         # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
#         # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
#         qkv = self.c_attn(x)
#         q, k, v = qkv.split(self.n_embd, dim=2)
#         k = k.view(B, T, self.n_head, C // self.n_head).transpose(
#             1, 2
#         )  # (B, nh, T, hs)
#         q = q.view(B, T, self.n_head, C // self.n_head).transpose(
#             1, 2
#         )  # (B, nh, T, hs)
#         v = v.view(B, T, self.n_head, C // self.n_head).transpose(
#             1, 2
#         )  # (B, nh, T, hs)
#         y = F.scaled_dot_product_attention(q, k, v, is_causal=True)  # flash attention
#         y = (
#             y.transpose(1, 2).contiguous().view(B, T, C)
#         )  # re-assemble all head outputs side by side

#         # output projection
#         y = self.c_proj(y)
#         return y


# class MLP(nn.Module):
#     def __init__(self, config) -> None:
#         super().__init__()
#         self.c_fc = nn.Linear(config.n_embd, config.n_embd * 4)
#         self.gelu = nn.GELU(approximate="tanh")
#         self.c_proj = nn.Linear(config.n_embd * 4, config.n_embd)
#         self.c_proj.GPT_SCALE = 1

#     def forward(self, x):
#         x = self.c_fc(x)
#         x = self.gelu(x)
#         x = self.c_proj(x)
#         return x


# class Block(nn.Module):
#     def __init__(self, config) -> None:
#         super().__init__()
#         self.ln_1 = nn.LayerNorm(config.n_embd)
#         self.attn = CausalSelfAttention(config)
#         self.ln_2 = nn.LayerNorm(config.n_embd)
#         self.mlp = MLP(config)

#         # attention is a communication -  aggregation function , a weigtes sum function a redcue function
#         # MLP is a individual function  - MAP function ( MAP reduce )

#     def forward(self, x):
#         x = x + self.attn(self.ln_1(x))  # self attention layer
#         x = x + self.mlp(self.ln_2(x))  # feed forward network
#         return x


# @dataclass
# class GPTConfig:
#     block_size: int = 1024
#     vocab_size: int = (
#         50304  # number of tokens : 50k + special tokens , overwrite the 50257 with this
#     )
#     n_layers: int = 12
#     n_head: int = 16
#     n_embd: int = 1024


# class GPT(nn.Module):
#     """
#     GPT (Generative Pre-trained Transformer) model class.

#     Args:
#         config (GPTConfig): Configuration object containing model hyperparameters.

#     Attributes:
#         config (GPTConfig): Configuration object containing model hyperparameters.
#         transformer (nn.ModuleDict): Dictionary containing the transformer components:
#             - wte (nn.Embedding): Token embedding layer.
#             - wpe (nn.Embedding): Position embedding layer.
#             - h (nn.ModuleList): List of transformer blocks.
#             - ln_f (nn.LayerNorm): Final layer normalization.
#         lm_head (nn.Linear): Linear layer for output logits.

#     Methods:
#         forward(idx):
#             Forward pass of the GPT model.
#             Args:
#                 idx (torch.Tensor): Input tensor of shape (B, T) where B is the batch size and T is the sequence length.
#             Returns:
#                 torch.Tensor: Output logits of shape (B, T, vocab_size).

#         from_pretrained(cls, model_type):
#             Load a pre-trained GPT model from Hugging Face.
#             Args:
#                 model_type (str): Type of the pre-trained model to load. Must be one of {"gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"}.
#             Returns:
#                 GPT: An instance of the GPT model with pre-trained weights.
#     """

#     def __init__(self, config) -> None:
#         super().__init__()
#         self.config = config

#         self.transformer = nn.ModuleDict(
#             dict(
#                 wte=nn.Embedding(config.vocab_size, config.n_embd),  # token embedding
#                 wpe=nn.Embedding(
#                     config.block_size, config.n_embd
#                 ),  # position embedding
#                 h=nn.ModuleList(
#                     [Block(config) for _ in range(config.n_layers)]
#                 ),  # transformer block
#                 ln_f=nn.LayerNorm(config.n_embd),  # final layer normalization
#             )
#         )
#         self.lm_head = nn.Linear(
#             config.n_embd, config.vocab_size, bias=False
#         )  # head for output
#         # weight sharing scheme
#         self.transformer.wte.weight = self.lm_head.weight

#     def _init_weight(self, module):
#         if isinstance(module, nn.Linear):
#             std = 0.02
#             if hasattr(module, "GPT_SCALE"):
#                 std *= (2 * self.config.n_layer) ** -0.5
#             torch.nn.init.normal_(module.weight, mean=0.0, std=std)
#             if module.bias is not None:
#                 torch.nn.init.zeros_(module.bias)

#             elif isinstance(module, nn.Embedding):
#                 torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

#     def forward(self, idx, targets=None):
#         # idx is of the size ( B , T )
#         B, T = idx.size()
#         assert (
#             T <= self.config.block_size
#         ), f"cannot forward the sequence length {T} , blocksize is less than the {T}"

#         pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
#         pos_emb = self.transformer.wpe(pos)
#         tok_emb = self.transformer.wte(idx)
#         x = (
#             tok_emb + pos_emb
#         )  # broadcasting is happening here as the positional emb will be same for each of the samples in the bacth
#         for block in self.transformer.h:
#             x = block(x)
#         x = self.transformer.ln_f(x)
#         logits = self.lm_head(x)  # (B , T , vocab_size)
#         loss = None
#         if targets is not None:
#             loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
#         return logits, loss
    
    


#     @classmethod
#     def from_pretrained(cls, model_type):
#         assert model_type in {"gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"}
#         from transformers import GPT2LMHeadModel

#         print("Loading model from Hugging Face: %s" % model_type)

#         config_args = {
#             "gpt2": dict(
#                 n_layers=12,
#                 n_embd=768,
#                 n_head=12,
#             ),
#             "gpt2-medium": dict(
#                 n_layers=24,
#                 n_embd=1024,
#                 n_head=16,
#             ),
#             "gpt2-large": dict(
#                 n_layers=36,
#                 n_embd=1280,
#                 n_head=20,
#             ),
#             "gpt2-xl": dict(
#                 n_layers=48,
#                 n_embd=1600,
#                 n_head=25,
#             ),
#         }[model_type]

#         config_args["vocab_size"] = 50257
#         config_args["block_size"] = 1024
#         config = GPTConfig(**config_args)

#         model = GPT(config)
#         sd = model.state_dict()
#         sd_keys = sd.keys()
#         sd_keys = [k for k in sd_keys if not k.endswith("attn.bias")]
#         model_hf = GPT2LMHeadModel.from_pretrained(model_type)
#         sd_hf = model_hf.state_dict()
#         sd_keys_hf = sd_hf.keys()
#         sd_keys_hf = [
#             k for k in sd_keys_hf if not k.endswith("attn.masked_bias")
#         ]  # remove the bias for attention
#         sd_keys_hf = [
#             k for k in sd_keys_hf if not k.endswith("attn.bias")
#         ]  # remove the bias for attention
#         transposed = [
#             "attn.c_attn.weight",
#             "attn.c_proj.weight",
#             "mlp.c_fc.weight",
#             "mlp.c_proj.weight",
#         ]
#         print(len(sd_keys_hf), len(sd_keys))
#         assert len(sd_keys_hf) == len(
#             sd_keys
#         ), f"mismatched keys : {len(sd_keys_hf)  != {len(sd_keys)}}"

#         for k in sd_keys_hf:
#             if any(k.endswith(w) for w in transposed):
#                 # special treatment for te conv1D weights we need to transpose the matrices
#                 assert sd_hf[k].shape[::-1] == sd[k].shape
#                 with torch.no_grad():
#                     sd[k].copy_(sd_hf[k].t())
#             else:
#                 # vanilla copy over the other parameters
#                 assert sd_hf[k].shape == sd[k].shape
#                 with torch.no_grad():
#                     sd[k].copy_(sd_hf[k])
#         return model
    
#     def configure_optimizer(self , weight_decay, learning_rate, epsilon):
#         # Create the optimizer
#         param_dict = {pm : p for pm , p in self.named_parameters() } 
#         param_dict = {k: v for k, v in param_dict.items() if v.requires_grad}
#         decay_param = [v for k, v in param_dict.items() if v.dim() >= 2]
#         nodecay_param = [v for k, v in param_dict.items() if v.dim()< 2 ]
#         optim_groups = [
#             {'params': decay_param, 'weight_decay': weight_decay},
#             {'params': nodecay_param, 'weight_decay': 0.0}
#         ]
#         num_decay_params =  sum(p.numel() for p in decay_param)
#         num_nodecay_params = sum(p.numel() for p in nodecay_param)
#         print(f"num decayed parameters tensor : {len(decay_param)} , with {num_decay_params} parametere " )
#         print(f"num non decayed parameters tensor : {len(nodecay_param)} , with {num_nodecay_params} parametere " )
#         fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
#         use_fused = fused_available and 'cuda' in device
#         optimizer = torch.optim.AdamW(
#             optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=epsilon ,fused=use_fused
#         )
#         return optimizer


# from torch.distributed import init_process_group , destroy_process_group

# # ddp = int(os.environ.get("RANK", -1)) != 1 
# ddp = None

# if ddp : 
#     assert torch.cuda.is_available()
#     init_process_group(backend="nccl")
#     ddp_rank = int(os.environ["RANK"])
#     ddp_local_rank = int(os.environ["LOCAL_RANK"])  
#     device = f"cuda:{ddp_local_rank}"
#     torch.cuda.set_device(device)
#     master_process = ddp_rank == 0
# else:
#     ddp_rank = 0
#     ddp_local_rank = 0  
#     ddp_world_size = 1  
#     master_process = True
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     print(f"using device : {device}")

# torch.manual_seed(1337)
# if torch.cuda.is_available():
#     device = "cuda"
#     torch.cuda.manual_seed(1337)

# import time

# model = GPT(GPTConfig())

# optimizer = model.configure_optimizer(weight_decay=0.1, learning_rate=6e-4, epsilon=1e-8)
# # model = torch.nn.DataParallel(model, device_ids=[0, 1]).to(device)
# model.to(device)
# model = torch.compile(model)
# # optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)

# import torch
# from torch.utils.data import IterableDataset, DataLoader
# from tokenizers import ByteLevelBPETokenizer
# import queue
# import threading
# from torch.cuda.amp import autocast, GradScaler


# class PrefetchingDatasetLoader(IterableDataset):
#     def __init__(self, file_path, tokenizer, block_size, chunk_size=1024 * 1024):
#         self.file_path = file_path
#         self.tokenizer = tokenizer
#         self.block_size = block_size
#         self.chunk_size = chunk_size

#     def _tokenize_chunk(self, chunk):
#         tokens = self.tokenizer.encode(chunk).ids
#         return tokens

#     def __iter__(self):
#         buffer = ""
#         tokens = []
#         chunk_queue = queue.Queue(maxsize=2)  # Set a small buffer for prefetching

#         # Start a background thread to read and tokenize chunks
#         def background_loader():
#             with open(self.file_path, "r") as f:
#                 while True:
#                     chunk = f.read(self.chunk_size)
#                     if not chunk:
#                         break
#                     # Tokenize and enqueue the chunk
#                     tokenized_chunk = self._tokenize_chunk(chunk)
#                     chunk_queue.put(tokenized_chunk)

#             chunk_queue.put(None)  # Signal that loading is done

#         threading.Thread(target=background_loader, daemon=True).start()

#         while True:
#             # Get the next tokenized chunk
#             tokenized_chunk = chunk_queue.get()
#             if tokenized_chunk is None:
#                 break  # End of file

#             tokens.extend(tokenized_chunk)

#             # Yield batches from tokens
#             while len(tokens) > self.block_size:
#                 x = torch.tensor(tokens[: self.block_size], dtype=torch.long)
#                 y = torch.tensor(tokens[1 : self.block_size + 1], dtype=torch.long)
#                 yield x, y
#                 tokens = tokens[self.block_size :]

#         # Yield any remaining tokens
#         if len(tokens) > self.block_size:
#             x = torch.tensor(tokens[: self.block_size], dtype=torch.long)
#             y = torch.tensor(tokens[1 : self.block_size + 1], dtype=torch.long)
#             yield x, y

        
# def next_batch(dataloader_iterator):
#     """
#     Fetch the next batch from the dataloader iterator.
#     Args:
#         dataloader_iterator: Iterator of the DataLoader.
#     Returns:
#         Tuple (x, y) containing the input and target tensors for the batch.
#         Returns None if the iterator is exhausted.
#     """
#     try:
#         return next(dataloader_iterator)
#     except StopIteration:
#         return None



# # Configuration
# B, T = 8, 1024  # micro Batch size and token sequence length

# total_batch_size = 524288
# assert total_batch_size % (B*T*ddp_world_size) == 0
# grad_accum_steps = total_batch_size//(B*T*ddp_world_size)
# if master_process:  
#     print(f"total desired batch size : {total_batch_size} , grad_accum_steps : {grad_accum_steps}")

# print("I am GPU ", ddp_rank)
# tokenizer = ByteLevelBPETokenizer(
#     "/kaggle/input/tokens-bytelevel/token_AAA/vocab.json",
#     "/kaggle/input/tokens-bytelevel/token_AAA/merges.txt",
# )

# # Create the dataset and dataloader
# dataset = PrefetchingDatasetLoader(
#     "/kaggle/input/python-code-regex/python_code.txt", tokenizer, block_size=T
# )
# dataloader = DataLoader(
#     dataset, batch_size=B, num_workers=4
# )  # Set `num_workers` for parallel loading

# # Training loop
# # Iterate through batches
# # for batch in dataloader:
# #     x, y = batch
# #     # Your training code here
# # scaler = torch.amp.GradScaler('cuda')

# max_lr = 6e-4
# min_lr = max_lr / 10
# warmup_steps = 20 
# max_steps = 100
# def get_lr(it):
#     if it< warmup_steps:
#         return max_lr * (it+1) / warmup_steps
#     if it > max_steps:
#         return min_lr
#     decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
#     assert 0.0 <= decay_ratio <= 1.0
#     coeff = 0.5 * (1.0 + torch.cos(torch.tensor(decay_ratio) * 3.14159))    
#     return min_lr + 0.5 * (max_lr - min_lr) * coeff

# use_amp = True
# scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
# loader = iter(dataloader)
# epoch_size = 0
# for epoch in range(2):  # Number of epochs
#     clips = epoch*max_steps
#     # for i, (x, y) in enumerate(dataloader):
#     for i in range(50):
#         optimizer.zero_grad()
#         t0 = time.time()
#         loss_accum = 0.0
#         for micro_step in range(grad_accum_steps):
#             x , y =  next_batch(loader)
#             x, y = x.to(device), y.to(device)
#             # print(x)
#             # print(y)
#             # print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
            
#             #         optimizer.zero_grad()

#             # Use autocast for mixed precision
#             with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
#                 logits, loss = model(x, y)
                
#             loss = loss / grad_accum_steps
#             loss_accum += loss.detach()
#             #         loss.mean().backward()
#             #         optimizer.step()
#             #         Scale the loss and backpropagate
#             scaler.scale(loss.mean()).backward()
#         norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         lr = get_lr(i + clips) 
#         for param_group in optimizer.param_groups:
#             param_group["lr"] = lr 
            
#         scaler.step(optimizer)
#         scaler.update()
#         #         opt.zero_grad() # set_to_none=True here can modestly improve performance

#         # ``scaler.step()`` first unscales the gradients of the optimizer's assigned parameters.
#         # If these gradients do not contain ``inf``s or ``NaN``s, optimizer.step() is then called,
#         # otherwise, optimizer.step() is skipped.
#         #         scaler.step(optimizer)

#         # Updates the scale for next iteration.
#         #         scaler.update()

#         torch.cuda.synchronize()
#         t1 = time.time()
#         dt = (t1 - t0) * 1000
#         tokens_per_sec = (B * T * grad_accum_steps / dt) * 1000
#         print(
#             f"step {i}, loss: {loss.mean()} , learnign rate : {lr} ,  norm : {norm} grads , dt : {dt} ms , tokens/sec : {tokens_per_sec}"
#         )
#         if i >= 50:
#             break
#     #     print(epoch_size)
#     #     epoch_size = epoch_size + 1
#     # print(epoch_size)
#     break

# # import sys

# # sys.exit(
# #     0
# # )  # ------------------------------------------------------------------------------------------------------------------------------------- #


# # model = GPT.from_pretrained("gpt2")
# # print("dtidtnt creashed")
# # num_return_sequence = 5
# # max_length = 30

# # enc = tiktoken.get_encoding("gpt2")
# # tokens = enc.encode("hello , I'm a language model")
# # print(tokens)
# # tokens = torch.tensor(tokens, dtype=torch.long)  # (8 ,)
# # tokens = tokens.unsqueeze(0).repeat(num_return_sequence, 1)  # (5 , 8)
# # x = tokens.to(device)

# # torch.manual_seed(42)
# # if device == "cuda":
# #     torch.cuda.manual_seed(42)
# # while x.size(1) < max_length:
# #     with torch.no_grad():
# #         logits = model(x)
# #         logits = logits[:, -1, :]  # (B , vocab_size)
# #         probs = F.softmax(logits, dim=-1)
# #         topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
# #         ix = torch.multinomial(topk_probs, 1)
# #         xcol = torch.gather(topk_indices, -1, ix)
# #         x = torch.cat((x, xcol), dim=1)

# # for i in range(num_return_sequence):
# #     tokens = x[i, :max_length].tolist()
# #     decoded = enc.decode(tokens)
# #     print(">", decoded)
# x

In [3]:
# Model working with torch.compile() -  T4 (but only on 1 GPU)

# !pip install tiktoken 
# !pip install triton 

# from dataclasses import dataclass
# import torch
# import torch.nn as nn
# from torch.nn import functional as F

# # prefix tokens
# import tiktoken
# import os
# import json
# from tokenizers import ByteLevelBPETokenizer
# torch.set_float32_matmul_precision('high')
# # # Set the CUDA_VISIBLE_DEVICES environment variable
# # # os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Use GPU 0
# # print("Available GPUs:", torch.cuda.device_count())
# # # import os
# # # import torch
 
# # # Clear CUDA environment setting
# # # os.environ["CUDA_VISIBLE_DEVICES"] = ""

# # # Reinitialize CUDA to clear prior states
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()

# # Attempt to do the autodetect of the device(GPU or CPU)

# device = "cpu"
# torch.manual_seed(1337)
# if torch.cuda.is_available():
#     device = "cuda"
#     torch.cuda.manual_seed(1337)

# elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
#     device = "mps"

# print(f"using device : {device}")


# # print("Available GPUs:", torch.cuda.device_count())


# class CausalSelfAttention(nn.Module):
#     def __init__(self, config) -> None:
#         super().__init__()
#         assert config.n_embd % config.n_head == 0

#         self.c_attn = nn.Linear(config.n_embd, config.n_embd * 3)
#         self.n_head = config.n_head
#         self.c_proj = nn.Linear(config.n_embd, config.n_embd)
#         self.n_embd = config.n_embd
#         self.c_proj.GPT_SCALE = 1
#         self.register_buffer(
#             "bias",
#             torch.tril(torch.ones(config.block_size, config.block_size)).view(
#                 1, 1, config.block_size, config.block_size
#             ),
#         )

#     def forward(self, x):
#         B, T, C = (
#             x.size()
#         )  # batch size, sequence length, embedding dimensionality (n_embd)
#         # calculate query, key, values for all heads in batch and move head forward to be the batch dim
#         # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
#         # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
#         qkv = self.c_attn(x)
#         q, k, v = qkv.split(self.n_embd, dim=2)
#         k = k.view(B, T, self.n_head, C // self.n_head).transpose(
#             1, 2
#         )  # (B, nh, T, hs)
#         q = q.view(B, T, self.n_head, C // self.n_head).transpose(
#             1, 2
#         )  # (B, nh, T, hs)
#         v = v.view(B, T, self.n_head, C // self.n_head).transpose(
#             1, 2
#         )  # (B, nh, T, hs)
#         y = F.scaled_dot_product_attention(q, k, v, is_causal=True)  # flash attention
#         y = (
#             y.transpose(1, 2).contiguous().view(B, T, C)
#         )  # re-assemble all head outputs side by side
#         # output projection
#         y = self.c_proj(y)
#         return y 

# class MLP(nn.Module):
#     def __init__(self, config) -> None:
#         super().__init__()
#         self.c_fc = nn.Linear(config.n_embd, config.n_embd * 4)
#         self.gelu = nn.GELU(approximate="tanh")
#         self.c_proj = nn.Linear(config.n_embd * 4, config.n_embd)
#         self.c_proj.GPT_SCALE = 1

#     def forward(self, x):
#         x = self.c_fc(x)
#         x = self.gelu(x)
#         x = self.c_proj(x)
#         return x

# class Block(nn.Module):
#     def __init__(self, config) -> None:
#         super().__init__()
#         self.ln_1 = nn.LayerNorm(config.n_embd)
#         self.attn = CausalSelfAttention(config)
#         self.ln_2 = nn.LayerNorm(config.n_embd)
#         self.mlp = MLP(config)

#         # attention is a communication -  aggregation function , a weigtes sum function a redcue function
#         # MLP is a individual function  - MAP function ( MAP reduce )

#     def forward(self, x):
#         x = x + self.attn(self.ln_1(x))  # self attention layer
#         x = x + self.mlp(self.ln_2(x))  # feed forward network
#         return x


# @dataclass
# class GPTConfig:
#     block_size: int = 1024
#     vocab_size: int = 50257  # number of tokens : 50k + special tokens
#     n_layers: int = 12
#     n_head: int = 12
#     n_embd: int = 768

# class GPT(nn.Module):
#     """
#     GPT (Generative Pre-trained Transformer) model class.

#     Args:
#         config (GPTConfig): Configuration object containing model hyperparameters.

#     Attributes:
#         config (GPTConfig): Configuration object containing model hyperparameters.
#         transformer (nn.ModuleDict): Dictionary containing the transformer components:
#             - wte (nn.Embedding): Token embedding layer.
#             - wpe (nn.Embedding): Position embedding layer.
#             - h (nn.ModuleList): List of transformer blocks.
#             - ln_f (nn.LayerNorm): Final layer normalization.
#         lm_head (nn.Linear): Linear layer for output logits.

#     Methods:
#         forward(idx):
#             Forward pass of the GPT model.
#             Args:
#                 idx (torch.Tensor): Input tensor of shape (B, T) where B is the batch size and T is the sequence length.
#             Returns:
#                 torch.Tensor: Output logits of shape (B, T, vocab_size).

#         from_pretrained(cls, model_type):
#             Load a pre-trained GPT model from Hugging Face.
#             Args:
#                 model_type (str): Type of the pre-trained model to load. Must be one of {"gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"}.
#             Returns:
#                 GPT: An instance of the GPT model with pre-trained weights.
#     """

#     def __init__(self, config) -> None:
#         super().__init__()
#         self.config = config

#         self.transformer = nn.ModuleDict(
#             dict(
#                 wte=nn.Embedding(config.vocab_size, config.n_embd),  # token embedding
#                 wpe=nn.Embedding(
#                     config.block_size, config.n_embd
#                 ),  # position embedding
#                 h=nn.ModuleList(
#                     [Block(config) for _ in range(config.n_layers)]
#                 ),  # transformer block
#                 ln_f=nn.LayerNorm(config.n_embd),  # final layer normalization
#             )
#         )
#         self.lm_head = nn.Linear(
#             config.n_embd, config.vocab_size, bias=False
#         )  # head for output
#         # weight sharing scheme 
#         self.transformer.wte.weight = self.lm_head.weight

#     def _init_weight(self, module):
#         if isinstance(module , nn.Linear):
#             std = 0.02
#             if hasattr(module , 'GPT_SCALE'):
#                 std *= (2*self.config.n_layer)**-0.5
#             torch.nn.init.normal_(module.weight , mean =0.0 , std = std)
#             if module.bias is not None : 
#                 torch.nn.init.zeros_(module.bias)
            
#             elif isinstance(module , nn.Embedding):
#                 torch.nn.init.normal_(module.weight , mean = 0.0 , std = 0.02)

#     def forward(self, idx, targets=None):
#         # idx is of the size ( B , T )
#         B, T = idx.size()
#         assert (
#             T <= self.config.block_size
#         ), f"cannot forward the sequence length {T} , blocksize is less than the {T}"

#         pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
#         pos_emb = self.transformer.wpe(pos)
#         tok_emb = self.transformer.wte(idx)
#         x = (
#             tok_emb + pos_emb
#         )  # broadcasting is happening here as the positional emb will be same for each of the samples in the bacth
#         for block in self.transformer.h:
#             x = block(x)
#         x = self.transformer.ln_f(x)
#         logits = self.lm_head(x)  # (B , T , vocab_size)
#         loss = None
#         if targets is not None:
#             loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
#         return logits, loss

#     @classmethod
#     def from_pretrained(cls, model_type):
#         assert model_type in {"gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"}
#         from transformers import GPT2LMHeadModel

#         print("Loading model from Hugging Face: %s" % model_type)

#         config_args = {
#             "gpt2": dict(
#                 n_layers=12,
#                 n_embd=768,
#                 n_head=12,
#             ),
#             "gpt2-medium": dict(
#                 n_layers=24,
#                 n_embd=1024,
#                 n_head=16,
#             ),
#             "gpt2-large": dict(
#                 n_layers=36,
#                 n_embd=1280,
#                 n_head=20,
#             ),
#             "gpt2-xl": dict(
#                 n_layers=48,
#                 n_embd=1600,
#                 n_head=25,
#             ),
#         }[model_type]

#         config_args["vocab_size"] = 50257
#         config_args["block_size"] = 1024
#         config = GPTConfig(**config_args)

#         model = GPT(config)
#         sd = model.state_dict()
#         sd_keys = sd.keys()
#         sd_keys = [k for k in sd_keys if not k.endswith("attn.bias")]
#         model_hf = GPT2LMHeadModel.from_pretrained(model_type)
#         sd_hf = model_hf.state_dict()
#         sd_keys_hf = sd_hf.keys()
#         sd_keys_hf = [
#             k for k in sd_keys_hf if not k.endswith("attn.masked_bias")
#         ]  # remove the bias for attention
#         sd_keys_hf = [
#             k for k in sd_keys_hf if not k.endswith("attn.bias")
#         ]  # remove the bias for attention
#         transposed = [
#             "attn.c_attn.weight",
#             "attn.c_proj.weight",
#             "mlp.c_fc.weight",
#             "mlp.c_proj.weight",
#         ]
#         print(len(sd_keys_hf), len(sd_keys))
#         assert len(sd_keys_hf) == len(
#             sd_keys
#         ), f"mismatched keys : {len(sd_keys_hf)  != {len(sd_keys)}}"

#         for k in sd_keys_hf:
#             if any(k.endswith(w) for w in transposed):
#                 # special treatment for te conv1D weights we need to transpose the matrices
#                 assert sd_hf[k].shape[::-1] == sd[k].shape
#                 with torch.no_grad():
#                     sd[k].copy_(sd_hf[k].t())
#             else:
#                 # vanilla copy over the other parameters
#                 assert sd_hf[k].shape == sd[k].shape
#                 with torch.no_grad():
#                     sd[k].copy_(sd_hf[k])
#         return model

# import time  

# model = GPT(GPTConfig())
# # model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)
# model.to(device)
# model = torch.compile(model)
# optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

# import torch
# from torch.utils.data import IterableDataset, DataLoader
# from tokenizers import ByteLevelBPETokenizer
# import queue
# import threading
# from torch.cuda.amp import autocast, GradScaler


# class PrefetchingDatasetLoader(IterableDataset):
#     def __init__(self, file_path, tokenizer, block_size, chunk_size=1024*1024):
#         self.file_path = file_path
#         self.tokenizer = tokenizer
#         self.block_size = block_size
#         self.chunk_size = chunk_size

#     def _tokenize_chunk(self, chunk):
#         tokens = self.tokenizer.encode(chunk).ids
#         return tokens

#     def __iter__(self):
#         buffer = ""
#         tokens = []
#         chunk_queue = queue.Queue(maxsize=2)  # Set a small buffer for prefetching

#         # Start a background thread to read and tokenize chunks
#         def background_loader():
#             with open(self.file_path, "r") as f:
#                 while True:
#                     chunk = f.read(self.chunk_size)
#                     if not chunk:
#                         break
#                     # Tokenize and enqueue the chunk
#                     tokenized_chunk = self._tokenize_chunk(chunk)
#                     chunk_queue.put(tokenized_chunk)

#             chunk_queue.put(None)  # Signal that loading is done

#         threading.Thread(target=background_loader, daemon=True).start()

#         while True:
#             # Get the next tokenized chunk
#             tokenized_chunk = chunk_queue.get()
#             if tokenized_chunk is None:
#                 break  # End of file

#             tokens.extend(tokenized_chunk)

#             # Yield batches from tokens
#             while len(tokens) > self.block_size:
#                 x = torch.tensor(tokens[:self.block_size], dtype=torch.long)
#                 y = torch.tensor(tokens[1:self.block_size + 1], dtype=torch.long)
#                 yield x, y
#                 tokens = tokens[self.block_size:]

#         # Yield any remaining tokens
#         if len(tokens) > self.block_size:
#             x = torch.tensor(tokens[:self.block_size], dtype=torch.long)
#             y = torch.tensor(tokens[1:self.block_size + 1], dtype=torch.long)
#             yield x, y

# # Configuration
# B, T = 8, 1024  # Batch size and sequence length
# tokenizer = ByteLevelBPETokenizer(
#     "/kaggle/input/tokens-bytelevel/token_AAA/vocab.json",
#     "/kaggle/input/tokens-bytelevel/token_AAA/merges.txt"
# )

# # Create the dataset and dataloader
# dataset = PrefetchingDatasetLoader("/kaggle/input/python-code-regex/python_code.txt", tokenizer, block_size=T)
# dataloader = DataLoader(dataset, batch_size=B, num_workers=4)  # Set `num_workers` for parallel loading

# # Training loop
# # Iterate through batches
# # for batch in dataloader:
# #     x, y = batch
# #     # Your training code here
# # scaler = torch.amp.GradScaler('cuda')
# use_amp= True
# scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
# epoch_size = 0
# for epoch in range(2):  # Number of epochs
#     for i, (x, y) in enumerate(dataloader):
#         t0 = time.time()
#         x, y = x.to(device), y.to(device)
#         # print(x)
#         # print(y)
#         # print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#         optimizer.zero_grad()
# #         optimizer.zero_grad()
        
#         # Use autocast for mixed precision
#         with torch.autocast(device_type=device, dtype=torch.float16 , enabled=use_amp):
#             logits, loss = model(x, y)
        
# #         loss.mean().backward()
# #         optimizer.step()
# #         Scale the loss and backpropagate
#         scaler.scale(loss.mean()).backward()
#         scaler.step(optimizer)
#         scaler.update()
# #         opt.zero_grad() # set_to_none=True here can modestly improve performance

#         # ``scaler.step()`` first unscales the gradients of the optimizer's assigned parameters.
#         # If these gradients do not contain ``inf``s or ``NaN``s, optimizer.step() is then called,
#         # otherwise, optimizer.step() is skipped.
# #         scaler.step(optimizer)

#         # Updates the scale for next iteration.
# #         scaler.update()
        
#         torch.cuda.synchronize()
#         t1 = time.time()
#         dt = (t1-t0)*1000
#         tokens_per_sec = (B*T/dt)*1000
#         print(f"step {i}, loss: {loss.mean()} , dt : {dt} ms , tokens/sec : {tokens_per_sec}")
#         if i >= 50:
#             break
#     #     print(epoch_size)
#     #     epoch_size = epoch_size + 1 
#     # print(epoch_size)
#     break
    
# # import sys

# # sys.exit(
# #     0
# # )  # ------------------------------------------------------------------------------------------------------------------------------------- #


# # model = GPT.from_pretrained("gpt2")
# # print("dtidtnt creashed")
# # num_return_sequence = 5
# # max_length = 30

# # enc = tiktoken.get_encoding("gpt2")
# # tokens = enc.encode("hello , I'm a language model")
# # print(tokens)pippi
# # tokens = torch.tensor(tokens, dtype=torch.long)  # (8 ,)
# # tokens = tokens.unsqueeze(0).repeat(num_return_sequence, 1)  # (5 , 8)
# # x = tokens.to(device)

# # torch.manual_seed(42)
# # if device == "cuda":
# #     torch.cuda.manual_seed(42)
# # while x.size(1) < max_length:
# #     with torch.no_grad():
# #         logits = model(x)
# #         logits = logits[:, -1, :]  # (B , vocab_size)
# #         probs = F.softmax(logits, dim=-1)
# #         topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
# #         ix = torch.multinomial(topk_probs, 1)
# #         xcol = torch.gather(topk_indices, -1, ix)
# #         x = torch.cat((x, xcol), dim=1)

# # for i in range(num_return_sequence):
# #     tokens = x[i, :max_length].tolist()
# #     decoded = enc.decode(tokens)
# #     print(">", decoded)

In [4]:
# import kagglehub

# kagglehub.login()

# # Replace with path to directory containing model files.
# LOCAL_MODEL_DIR = 'path/to/files'

# MODEL_SLUG = 'my_model' # Replace with model slug.

# # Learn more about naming model variations at
# # https://www.kaggle.com/docs/models#name-model.
# VARIATION_SLUG = 'default' # Replace with variation slug.

# kagglehub.model_upload(
#   handle = f"deeperisbetter/{MODEL_SLUG}/pyTorch/{VARIATION_SLUG}",
#   local_model_dir = LOCAL_MODEL_DIR,
#   version_notes = 'Update 2024-11-16')

In [5]:
import os
import numpy as np
import tiktoken
from datasets import load_dataset
from tqdm import tqdm

# ------------------------------------------
local_dir = "edu_fineweb10B"
remote_name = "sample-10BT"
shard_size = int(1e8)  # 100M tokens per shard
num_shards_to_download = 95 # Limit to 8 shards per run
offset = 0  # Adjust this value for subsequent runs

# Create the local directory if it doesn't exist yet
DATA_CACHE_DIR = os.path.join(os.getcwd(), local_dir)
os.makedirs(DATA_CACHE_DIR, exist_ok=True)

# Load the dataset
fw = load_dataset("HuggingFaceFW/fineweb-edu", name=remote_name, split="train")

# Initialize the tokenizer
enc = tiktoken.get_encoding("gpt2")
eot = enc._special_tokens['<|endoftext|>' ]  # End of text token

def tokenize(doc):
    tokens = [eot]  # The special <|endoftext|> token delimits all documents
    tokens.extend(enc.encode_ordinary(doc["text"]))
    tokens_np = np.array(tokens, dtype=np.uint16)
    return tokens_np

def write_datafile(filename, tokens_np):
    np.save(filename, tokens_np)

# Get already downloaded files to avoid duplicates
downloaded_files = {f for f in os.listdir(DATA_CACHE_DIR) if f.endswith(".npy")}

# Tokenize and save only the required shards
current_shard = 0
token_count = 0
progress_bar = None
all_tokens_np = np.empty((shard_size,), dtype=np.uint16)  # Preallocate buffer for tokens

# Process shards based on offset and limit
for doc_index, doc in enumerate(tqdm(fw, desc="Processing Documents")):
    if current_shard < offset:
        continue  # Skip shards up to the offset

    # Tokenize the document
    tokens = tokenize(doc)

    # Check if we need to start a new shard
    if token_count + len(tokens) > shard_size:
        # Write the current shard to disk
        filename = os.path.join(DATA_CACHE_DIR, f"shard_{current_shard:06d}.npy")
        if os.path.basename(filename) in downloaded_files:
            print(f"Shard {filename} already exists, skipping.")
        else:
            write_datafile(filename, all_tokens_np[:token_count])
            downloaded_files.add(os.path.basename(filename))
            print(f"Saved {filename}")
        
        # Reset buffer for the next shard
        token_count = 0
        current_shard += 1

        # Stop if we've processed enough shards for this run
        if current_shard >= offset + num_shards_to_download:
            print(f"Downloaded {num_shards_to_download} shards. Exiting.")
            break

    # Add tokens to the buffer
    all_tokens_np[token_count:token_count+len(tokens)] = tokens
    token_count += len(tokens)

# Save any remaining tokens as the last shard
if token_count > 0 and current_shard < offset + num_shards_to_download:
    filename = os.path.join(DATA_CACHE_DIR, f"shard_{current_shard:06d}.npy")
    if os.path.basename(filename) not in downloaded_files:
        write_datafile(filename, all_tokens_np[:token_count])
        print(f"Saved final shard {filename}")


README.md:   0%|          | 0.00/23.3k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/1630 [00:00<?, ?it/s]

000_00000.parquet:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

001_00000.parquet:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

002_00000.parquet:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

...

Saved /kaggle/working/edu_fineweb10B/shard_000092.npy


Processing Documents:  94%|█████████▍| 9135056/9672101 [3:11:23<22:16, 401.95it/s]

Saved /kaggle/working/edu_fineweb10B/shard_000093.npy


Processing Documents:  95%|█████████▌| 9232458/9672101 [3:13:25<09:12, 795.54it/s]

Saved /kaggle/working/edu_fineweb10B/shard_000094.npy
Downloaded 95 shards. Exiting.
